## This notebook is meant to learn generative AI using Auto-encoders and GANs

* Generative Adverserial Networks
* Autoencoders

In [4]:
#import libraries deep learning based models
import keras
from keras import Input, layers
from keras.models import Model
from keras import backend as K

#importing models that are not deep learning based
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
latent_dim = 2
batch_size = 16

#this is the encoder
input_img = layers.Input(shape = (28,28,1))
x = layers.Conv2D(32,3,activation='relu',padding='same')(input_img)
x = layers.Conv2D(64,3,activation='relu', padding='same',strides=(2,2))(x)
x = layers.Conv2D(64,3,activation='relu',padding='same')(x)
x = layers.Conv2D(64,3,activation='relu', padding='same')(x)
dimension =K.int_shape(x)
x = layers.Flatten()(x)
x = layers.Dense(32, activation='relu')(x)

z_mean = layers.Dense(latent_dim)(x)
z_log_variance = layers.Dense(latent_dim)(x)

In [17]:
dimension

(None, 14, 14, 64)

In [7]:
K.shape(z_mean)[0]

<tf.Tensor 'strided_slice:0' shape=() dtype=int32>

In [14]:
#now to build that latent_space sampling function
def sampling(args):
    z_mean, z_log_variance = args
    epsilon = K.random_normal((K.shape(z_mean)[0], latent_dim))
    return z_mean + K.exp(z_log_variance)*epsilon

z = layers.Lambda(sampling)([z_mean, z_log_variance])

In [15]:
K.shape(z)

<tf.Tensor 'Shape_2:0' shape=(2,) dtype=int32>

In [16]:
K.int_shape(z)

(None, 2)

In [18]:
np.prod([2,3,4])

24

In [20]:
#building the decoder network, turning points on the sample spac to images
decoder_input = layers.Input(K.int_shape(z)[1:])
x = layers.Dense(np.prod(dimension[1:]), activation='relu')(decoder_input)
x = layers.Reshape(dimension[1:])(x)
x = layers.Conv2DTranspose(32,3,activation='relu',padding='same',strides=(2,2))(x)
x = layers.Conv2D(1,3,padding='same',activation='sigmoid')(x)

decoder = Model(decoder_input, x)
z_decoded = decoder(z)

In [23]:
class CustomVariationalLayer(keras.layers.Layer):
    
    def vae_loss(self, x, z_decoded):
        x = K.flatten(x)
        z_decoded = K.flatten(z_decoded)
        xent_loss = keras.metrics.binary_crossentropy(x, z_decoded)
        kl_loss = -5e-4 * K.mean(
        1 + z_log_variance - K.square(z_mean) - K.exp(z_log_variance), axis=-1)
        return K.mean(xent_loss + kl_loss)
    
    def call(self, inputs):
        x = inputs[0]
        z_decoded = inputs[1]
        loss = self.vae_loss(x, z_decoded)
        self.add_loss(loss, inputs=inputs)
        return x

y = CustomVariationalLayer()([input_img, z_decoded])